In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76543: Some(http://gw02.itversity.com:4045)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76543

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet"

path = /user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet


/user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet

In [6]:
// in Scala
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"
  .split(" ")
val words = spark.sparkContext.parallelize(myCollection, 2)

myCollection = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)
words = ParallelCollectionRDD[0] at parallelize at <console>:30


ParallelCollectionRDD[0] at parallelize at <console>:30

In [7]:
// COMMAND ----------

// in Scala
val supplementalData = Map("Spark" -> 1000, "Definitive" -> 200,
                           "Big" -> -300, "Simple" -> 100)

supplementalData = Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)


Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)

In [8]:
// COMMAND ----------

// in Scala
val suppBroadcast = spark.sparkContext.broadcast(supplementalData)

suppBroadcast = Broadcast(0)


Broadcast(0)

In [9]:
// COMMAND ----------

// in Scala
suppBroadcast.value


// COMMAND ----------

Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)

In [10]:
// in Scala
words.map(word => (word, suppBroadcast.value.getOrElse(word, 0)))
  .sortBy(wordPair => wordPair._2)
  .collect()

[(Big,-300), (The,0), (Guide,0), (:,0), (Data,0), (Processing,0), (Made,0), (Simple,100), (Definitive,200), (Spark,1000)]

In [11]:
// COMMAND ----------

// in Scala
case class Flight(DEST_COUNTRY_NAME: String,
                  ORIGIN_COUNTRY_NAME: String, count: BigInt)

defined class Flight


In [12]:
val flights = spark.read
  .parquet(path)
  .as[Flight]


// COMMAND ----------

flights = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [13]:
// in Scala
import org.apache.spark.util.LongAccumulator
val accUnnamed = new LongAccumulator
val acc1 = spark.sparkContext.register(accUnnamed)

accUnnamed = LongAccumulator(id: 100, name: None, value: 0)


acc1: Unit = ()


LongAccumulator(id: 100, name: None, value: 0)

In [14]:
// COMMAND ----------

// in Scala
val accChina = new LongAccumulator
val accChina2 = spark.sparkContext.longAccumulator("China2")
spark.sparkContext.register(accChina, "China")

accChina = LongAccumulator(id: 102, name: Some(China), value: 0)
accChina2 = LongAccumulator(id: 101, name: Some(China2), value: 0)


LongAccumulator(id: 101, name: Some(China2), value: 0)

In [15]:
// COMMAND ----------

// in Scala
def accChinaFunc(flight_row: Flight) = {
  val destination = flight_row.DEST_COUNTRY_NAME
  val origin = flight_row.ORIGIN_COUNTRY_NAME
  if (destination == "China") {
    accChina.add(flight_row.count.toLong)
  }
  if (origin == "China") {
    accChina.add(flight_row.count.toLong)
  }
}

accChinaFunc: (flight_row: Flight)Unit


In [16]:
// COMMAND ----------

// in Scala
flights.foreach(flight_row => accChinaFunc(flight_row))

In [17]:
// COMMAND ----------

// in Scala
accChina.value // 953

953

In [38]:
// COMMAND ----------

// in Scala
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.util.AccumulatorV2

//val arr = ArrayBuffer[BigInt]()

class EvenAccumulator extends AccumulatorV2[BigInt, BigInt] {
  private var num:BigInt = 0
  def reset(): Unit = {
    this.num = 0
  }
  def add(intValue: BigInt): Unit = {
    if (intValue % 2 == 0) {
        this.num += intValue
    }
  }
  def merge(other: AccumulatorV2[BigInt,BigInt]): Unit = {
    this.num += other.value
  }
  def value():BigInt = {
    this.num
  }
  def copy(): AccumulatorV2[BigInt,BigInt] = {
    new EvenAccumulator
  }
  def isZero():Boolean = {
    this.num == 0
  }
}

val acc = new EvenAccumulator
val newAcc = sc.register(acc, "evenAcc")

println(acc.value) // 0
flights.foreach(flight_row => acc.add(flight_row.count))
println(acc.value) // 31390

0


lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Exception while getting task result: java.io.IOException: java.lang.ClassCastException: cannot assign instance of scala.collection.immutable.List$SerializationProxy to field org.apache.spark.sql.catalyst.expressions.objects.StaticInvoke.arguments of type scala.collection.Seq in instance of org.apache.spark.sql.catalyst.expressions.objects.StaticInvoke
StackTrace:   at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1587)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1586)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1586)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:831)
  at org.apach